# Audio generation with a pipeline

Ses üretimi, bir ses çıktısı üretmeyi içeren çok yönlü bir dizi görevi kapsar. Burada inceleyeceğimiz görevler konuşma üretimi (diğer adıyla "metinden konuşmaya") ve müzik üretimidir. Metin-konuşmada, bir model bir metin parçasını gerçeğe yakın konuşma dili sesine dönüştürerek sanal asistanlar, görme engelliler için erişilebilirlik araçları ve kişiselleştirilmiş sesli kitaplar gibi uygulamalara kapı açar. Öte yandan, müzik üretimi yaratıcı ifadeyi mümkün kılabilir ve çoğunlukla eğlence ve oyun geliştirme endüstrilerinde kullanım alanı bulur.

Transformers'ta bu görevlerin her ikisini de kapsayan bir pipeline bulacaksınız. Bu pipeline "text-to-audio" olarak adlandırılır, ancak kolaylık sağlamak için "text-to-speech" takma adı da vardır. Burada her ikisini de kullanacağız ve siz göreviniz için hangisinin daha uygun olduğunu seçmekte özgürsünüz.

Sadece birkaç satır kodla metinler için sesli anlatım ve müzik üretmeye başlamak için bu pipeline'ı nasıl kullanabileceğinizi keşfedelim.

Bu pipeline Transformers için yenidir ve 4.32 sürümünün bir parçasıdır. Bu nedenle, özelliği elde etmek için kütüphaneyi en son sürüme yükseltmeniz gerekir:

In [1]:
!pip install -q -U transformers

## Generating speech

Metinden konuşmaya üretimi inceleyerek başlayalım. İlk olarak, tıpkı ses sınıflandırma ve otomatik konuşma tanımada olduğu gibi, pipeline'ı tanımlamamız gerekecek. Görevimizi en iyi şekilde tanımladığı için bir metinden konuşmaya pipeline tanımlayacağız ve suno/bark-small kontrol noktasını kullanacağız:

In [2]:
from transformers import pipeline

pipe = pipeline(task="text-to-speech", model="suno/bark-small", device="cuda")

2024-08-08 07:57:32.709640: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-08 07:57:32.709790: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-08 07:57:32.851438: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/8.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/opt/conda/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_b

generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
text = "Ladybugs have had important roles in culture and religion, being associated with luck, love, fertility and prophecy. "
output = pipe(text)
output

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'audio': array([[0.00152142, 0.00144757, 0.00158531, ..., 0.13999127, 0.1511539 ,
         0.16595556]], dtype=float32),
 'sampling_rate': 24000}

In [4]:
from IPython.display import Audio

Audio(output["audio"], rate=output["sampling_rate"])

Pipeline ile kullandığımız Bark modeli aslında çok dillidir, bu nedenle başlangıç metnini örneğin Fransızca bir metinle kolayca değiştirebilir ve pipeline'ı aynı şekilde kullanabiliriz. Dili kendi kendine algılayacaktır:

In [5]:
fr_text = "Contrairement à une idée répandue, le nombre de points sur les élytres d'une coccinelle ne correspond pas à son âge, ni en nombre d'années, ni en nombre de mois. "
output = pipe(fr_text)
Audio(output["audio"], rate=output["sampling_rate"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


Bu model sadece çok dilli olmakla kalmaz, aynı zamanda sözsüz iletişim ve şarkı söyleyerek ses üretebilir. İşte nasıl şarkı söyletebileceğinizi:

In [6]:
song = "♪ In the jungle, the mighty jungle, the ladybug was seen. ♪ "
output = pipe(song)
Audio(output["audio"], rate=output["sampling_rate"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.


## Generating music

In [7]:
music_pipe = pipeline(
    task="text-to-audio", model="facebook/musicgen-small", device="cuda"
)

config.json:   0%|          | 0.00/7.87k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
text = "90s rock song with electric guitar and heavy drums"
forward_params = {"max_new_tokens": 1024}

output = music_pipe(text, forward_params=forward_params)
Audio(output["audio"][0], rate=output["sampling_rate"])